In [40]:
#pip install flask

In [41]:
from flask import Flask, jsonify,abort
import requests
from bs4 import BeautifulSoup

In [42]:
app = Flask(__name__)

# Dummy ML function for sentiment analysis
def dummy_ml_function(text):
    # Replace this function with actual ML logic
    return "Positive"

In [43]:
def scrape_deepmind_publications():
    url = 'https://deepmind.com/research/publications/'
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the unordered list with publications
        publication_list = soup.find('ul', {'class': 'list-compact'})
        
        # Each publication is in a list item within the unordered list
        publications = publication_list.find_all('li', recursive=False)
        
        publication_data = []
        
        for publication in publications:
            # Extract date, title, authors, and venue
            date = publication.find('time').text.strip()
            title = publication.find('span', {'class': 'list-compact__inner'}).text.strip()
            authors = publication.find('dd', {'class': 'glue-caption'}).text.strip()
            venue = publication.find_all('dd', {'class': 'glue-caption'})[1].text.strip()
            
            publication_data.append({
                'date': date,
                'title': title,
                'authors': authors,
                'venue': venue
            })
        
        return publication_data
    
    else:
        print('Failed to retrieve the webpage')
        return []

# Call the function and print results
publications = scrape_deepmind_publications()
for pub in publications:
    print(pub)


{'date': '17 Jun 24\n17 June 2024', 'title': 'Mirasol3B: A Multimodal Autoregressive Model for Time-Aligned and Contextual Modalities', 'authors': 'AJ Piergiovanni, Isaac Noble, Dahun Kim, Michael S. Ryoo, Victor Gomes, Anelia Angelova', 'venue': 'CVPR 2024'}
{'date': '17 Jun 24\n17 June 2024', 'title': "Bayes' Rays: uncertainty quantification for neural radiance fields", 'authors': 'Lily Goli, Cody Reading, Silvia Sellan, Alec Jacobson, Andrea Tagliasacchi', 'venue': 'CVPR 2024'}
{'date': '17 Jun 24\n17 June 2024', 'title': 'Neural Fields as Distributions: Signal Processing Beyond Euclidean Space', 'authors': 'Daniel Rebain, Soroosh Yazdani, Kwang Moo Yi, Andrea Tagliasacchi', 'venue': 'CVPR 2024'}
{'date': '7 May 24\n7 May 2024', 'title': 'Teach LLMs to Phish: Stealing Private Information from Language Models', 'authors': 'Ashwinee Panda, Christopher A. Choquette-Choo, Zhengming Zhang, Yaoqing Yang, Prateek Mittal', 'venue': 'ICLR 2024'}
{'date': '7 May 24\n7 May 2024', 'title': 'Pri

In [44]:
# 主页端点
#@app.route('/')
#def home():
    #return "Welcome to the DeepMind News API!"


In [45]:
# Endpoint to fetch a list of articles
@app.route('/get_data', methods=['GET'])
def get_data():
    articles = scrape_deepmind_publications()
    # Just return the first 5 articles for brevity
    return jsonify(articles[:5])



In [46]:
# Endpoint to display information about articles
@app.route('/articles', methods=['GET'])
def get_articles():
    articles = scrape_deepmind_publications()
    # Transform articles for display, excluding content
    display_articles = [{'number': i+1, 'title': article['title'], 'date': article['date'], 'authors': article['authors'], 'venue': article['venue']} for i, article in enumerate(articles)]
    return jsonify(display_articles)

In [47]:
# Endpoint to access the content of a specified article
@app.route('/article/<int:number>', methods=['GET'])
def get_article(number):
    articles = scrape_deepmind_publications()
    # Transform articles for display, excluding content
    display_articles = [{'number': i+1, 'title': article['title'], 'date': article['date'], 'authors': article['authors'], 'venue': article['venue']} for i, article in enumerate(articles)]
    article = next((item for item in display_articles if item['number'] == number), None)
    if article is not None:
        return jsonify(article)
    else:
        abort(404, description="Resource not found")


In [48]:
# Endpoint for machine learning analysis on all articles or a single one
@app.route('/ml', defaults={'number': None})
@app.route('/ml/<int:number>', methods=['GET'])
def ml_analysis(number):
    articles = scrape_deepmind_publications()
    display_articles = [{'number': i+1, 'title': article['title'], 'date': article['date'], 'authors': article['authors'], 'venue': article['venue']} for i, article in enumerate(articles)]
    if number is None:
        # Apply ML function to all articles
        results = {article['number']: dummy_ml_function(article['title']) for article in display_articles}
    else:
        # Apply ML function to the specific article
        article = next((item for item in display_articles if item['number'] == number), None)
        if article is not None:
            results = {number: dummy_ml_function(article['title'])}
        else:
            abort(404, description="Resource not found")
    return jsonify(results)

In [49]:
from threading import Thread

def run_app():
    app.run()

flask_thread = Thread(target=run_app)
flask_thread.start()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production


In [ ]:
#if __name__ == '__main__':
   # app.run(debug=True)

[2024-04-05 13:22:53,979] ERROR in app: Exception on /article/1 [GET]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-17-84cbbc9dde42>", line 5, in get_article
    article = ne

 * Debug mode: off


127.0.0.1 - - [05/Apr/2024 13:29:04] "GET /articles/1 HTTP/1.1" 404 -
127.0.0.1 - - [05/Apr/2024 13:30:53] "GET /articles/ HTTP/1.1" 404 -
127.0.0.1 - - [05/Apr/2024 13:30:59] "GET /articles HTTP/1.1" 200 -
[2024-04-05 13:31:36,445] ERROR in app: Exception on /article/1 [GET]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/anaconda3/lib/python3

   Use a production WSGI server instead.


127.0.0.1 - - [05/Apr/2024 14:11:39] "GET /articles HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2024 14:11:58] "GET /article/1 HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2024 14:12:19] "GET /ml HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2024 14:12:30] "GET /ml/1 HTTP/1.1" 200 -
